In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tqdm

%matplotlib inline

In [2]:
import matplotlib as mpl

In [3]:
import vehicleDetection

/home/tsbertalan/.virtualenvs/carnd/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from vehicleDetection.temporal import HeatVideo, DetectedVideo

# Load and look at the training data.

In [5]:
images, classes = vehicleDetection.data.getData(numLighter=0, numFlip=0)

HBox(children=(IntProgress(value=0, max=6726), HTML(value='')))

# Train the classifier on the full dataset with the best classifier parameters.

In [6]:
detectorNoAug = vehicleDetection.detector.Detector()

Got scalar parameter (C). Assuming no CV requested.


In [7]:
%%time

indices = np.arange(len(images))
np.random.shuffle(indices)
n = 1024
imageWindows = [images[i] for i in indices[:n]]
windowClasses = [classes[i] for i in indices[:n]]


detectorNoAug.fit(
    imageWindows,
    windowClasses,
)

/home/tsbertalan/.virtualenvs/carnd/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


920-image train accuracy of clf =  0.954347826087
104-image test accuracy of clf =  0.932692307692
CPU times: user 12.6 s, sys: 76 ms, total: 12.7 s
Wall time: 12.7 s


In [8]:
# %%time
# detectorNoAug.fit(
#     images,
#     classes,
# )

# Run the detector on a test image.

In [9]:
testh = HeatVideo()
projh = HeatVideo(fpath='../../../data/vehicleDetection/project_video.mp4', maxFrames=12)

In [10]:
test = DetectedVideo()
proj = DetectedVideo(
    fpath='../../../data/vehicleDetection/project_video.mp4',
)

In [11]:
testh.go(detectorNoAug, label='-unaugmented');

HBox(children=(IntProgress(value=0, description='heating: ', max=38), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', description='test_video-heat-unaugmented.mp4: ', max=1), HTML(value='')))

/home/tsbertalan/.virtualenvs/carnd/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


# Use exponential decay for heat persistence

In [15]:
testh.heatDecay();

HBox(children=(IntProgress(value=0, description='heatDecay.mp4: ', max=39), HTML(value='')))

In [16]:
import gc
gc.collect()

0

In [19]:
projh.go(detectorNoAug, label='-unaugmented');

HBox(children=(IntProgress(value=0, description='heating: ', max=12), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', description='project_video-heat-unaugmented.mp4: ', max=1), HTML(value='')))

/home/tsbertalan/.virtualenvs/carnd/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [20]:
projh.heatDecay();

HBox(children=(IntProgress(value=0, description='heatDecay.mp4: ', max=13), HTML(value='')))

In [ ]:
test.go(detectorNoAug, label='-unaugmented')

In [ ]:
proj.go(detectorNoAug, label='-unaugmented')

Try again on the agumented data.

In [ ]:
images, classes = vehicleDetection.data.getData()

In [ ]:
detectorAug = vehicleDetection.detector.Detector()

In [ ]:
%%time
detectorAug.fit(
    images,
    classes,
)

In [ ]:
testha = HeatVideo()
testha.go(detectorAug, label='-augmented')

In [ ]:
projha = HeatVideo(fpath='../../../data/vehicleDetection/project_video.mp4',)
projha.go(detectorAug, label='-augmented')

In [ ]:
testa = DetectedVideo(
    label='-augmented',
)
testa.go(detectorAug, label='-augmented')

In [ ]:
proja = DetectedVideo(
    fpath='../../../data/vehicleDetection/project_video.mp4',
)
proja.go(detectorAug, label='-augmented')